# Tarea 2 - Named Entity Recognition

----------------------

- **Nombre:**
Joaquin Peréz, Valentina Sepúlveda.
- **Usuario o nombre de equipo en Codalab:** 
vsepulv, meperd0nas

### Reglas de la tarea

Algunos detalles de la competencia:

- Para que su tarea sea evaluada, deben participar en la competencia como también, enviar este notebook con su informe.
- Para participar, deben registrarse en la competencia en Codalab en grupos de máximo 2 alumnos. Cada grupo debe tener un nombre de equipo. (¡Y deben reportarlo en su informe!)
- Las métricas usadas serán Precisión, Recall y F1.
- En esta tarea se recomienda usar GPU. Pueden ejecutar su tarea en colab (lo cual trae todo instalado) o pueden intentar correrlo en su computador. en este caso, deberá ser compatible con cuda y deberán instalar todo por su cuenta.
- En total pueden hacer un **máximo de 4 envíos**.
- Por favor, todas sus dudas haganlas en el hilo de U-cursos de la tarea. Los emails que lleguen al equipo docente serán remitidos a ese medio. Recuerden el ánimo colaborativo del curso!!
- Estar top 5 en alguna métrica equivale a 1 punto extra en la nota final.


**Link a la competencia:  https://competitions.codalab.org/competitions/25302?secret_key=690406c7-b3b0-4092-8694-d08d7991ca94**

### Reporte

Este debe cumplir la siguiente estructura:

1.	**Introducción**: Presentar brevemente el problema a resolver, los modelos utilizados en el desarrollo de la tarea y conclusiones obtenidas. (0.5 Puntos)

2.	**Modelos**: Describir brevemente los modelos, métodos y hiperparámetros utilizados. (1.0 puntos)

4.	**Métricas de evaluación**: Describir las métricas utilizadas en la evaluación indicando que miden y cuál es su interpretación en este problema en particular. (0.5 puntos)

5.	**Experimentos**: Reportar todos sus experimentos y código en esta sección. Comparar los resultados obtenidos utilizando diferentes modelos. ¡Es vital haber realizado varios experimentos para sacar una buena nota! (3.0 puntos)

6.	**Conclusiones**: Discutir resultados, proponer trabajo futuro. (1.0 punto)


-----------------------------------------

## Introducción

Dentro de los problemas del procesamiento del lenguaje natural, hay tres grandes categorias: *Sequence Classification*, *Sequence Labeling* y por el ultimo el *Sequence to Sequence*. El primero consiste en dado una oración ser capaz de clasificarlo en una clase discreta, ya sea por ejemplo de carga emocional positiva o negativa, o clasificarlo por el tema que trata.

El segundo, *Sequence Labeling*, trata que dado una oración, clasificar cada palabra en una categoria o etiqueta, dando como resultado una sequencia de etiquetas final del mismo largo que la inicial. Y por ultimo el *Sequence to Sequence* habla de transformar una oración en otra oración pero manteniendo sus propiedades.

De los tres tipos de problemas nos enfocaremos en el segundo, *Sequence Labeling*.
De aqui hay un problema conocido: el Named Entity Recognition del cual trata este trabajo. El *Named Entity Recognition* (o NER), en español traducido como Detección de Entidades trata de ser capaz de encontrar las entidades, ya sea una Empresa, Persona u otro tipo de categoria; dentro de una oración y marcarlas dentro de este input. Así cada palabra del input tiene una etiqueta asociada que explica cual Entidad tiene si es que tiene una, o si no tiene.

Para resolver este tipo de problemas se decidió resolverlos con un tipo de redes neuronales, las redes neuronales recurrentes (o RNN). Estas redes consisten en que definen sus valores actuales en base a sus valores pasados, así siendo capaz de reconstruir toda la secuencia y ser capaz de tomar en consideración el contexto al momento de clasificar.

Para esto definimos tres modelos, dos basados en un tipo de arquitectura de RNN distinta, el primer modelo basado en la arquitectura LSTM, el segundo basado en la arquitectura GRU. El utlimo es sin embargo otro tipo de modelo, una red neuronal convolucional (o CNN) simple.

Dados estos modelos, como resultados se obtuvo que...

In [ ]:
# Instalar torchtext (en codalab) - Descomentar.
!pip3 install torchtext==0.6
# !pip install transformers

In [2]:
%%capture
!wget https://github.com/dccuchile/CC6205/releases/download/Data/train_NER_esp.txt -nc # Dataset de Entrenamiento
!wget https://github.com/dccuchile/CC6205/releases/download/Data/val_NER_esp.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
!wget https://github.com/dccuchile/CC6205/releases/download/Data/test_NER_esp.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!


In [3]:
import torch
from torchtext import data, datasets

# Garantizar reproducibilidad 
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)

fields = (("text", TEXT), ("nertags", NER_TAGS))

train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="train_NER_esp.txt",
    validation="val_NER_esp.txt",
    test="test_NER_esp.txt",
    fields=fields,
    encoding="iso-8859-1",
    separator=" "
)
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']

BATCH_SIZE = 16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)


Using cuda


## Métricas de evaluación

- **Precision:** Esta métrica contabiliza la cantidad de verdaderos positivos (tp) versus la cantidad de falsos positivos (fp), con la siguiente fórmula: $\frac{tp}{tp+fp}$.

- **Recall:** Esta métrica considera, a diferencia de la "Precision" la cantidad de verdaderos positivos versus la cantidad de falsos negativos (fn), con la siguiente fórmula: $\frac{tp}{tp+fn}$.

- **F1 score:** El F1 score es la media armónica entre la presición y el recall, es decir: $2 \cdot \frac{precition \cdot recall}{precition + recall}$

In [4]:
# Definimos las métricas

from sklearn.metrics import f1_score, precision_score, recall_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore",
                        category=sklearn.exceptions.UndefinedMetricWarning)


def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)
    # Obtenemos los indices distintos de 0.

    # filtramos <pad> y O para calcular los scores.
    mask = [(y_true != o_idx) & (y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu')
    y_true = y_true.to('cpu')
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    return precision, recall, f1

-------------------

### Modelo Baseline

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir la época de entrenamiento
3. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]

        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
         nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [7]:
class GENERAL_NER_RNN(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx,
                 rnn_type="LSTM"):
        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        #Capa RNN
        #"""
        if rnn_type is "LSTM":
          self.rnn = nn.LSTM(embedding_dim,
                        hidden_dim,
                        num_layers=n_layers,
                        bidirectional=bidirectional,
                        dropout= dropout if n_layers> 1 else 0)
        elif rnn_type is "GRU":
          self.rnn = nn.GRU(embedding_dim,
                        hidden_dim,
                        num_layers=n_layers,
                        bidirectional=bidirectional,
                        dropout= dropout if n_layers> 1 else 0)      
        #"""
        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))

        outputs, (hidden) = self.rnn(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]
        return predictions



In [8]:
class GENERAL_NER_RNN_EMB(nn.Module):
    def __init__(self,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx,
                 rnn_type="LSTM"):

        super().__init__()

        # Capa de embedding
        self.embedding = external_embedding
        self.embedding.padding_idx = pad_idx
        #Capa GRU
        #"""
        if rnn_type == "LSTM":
          self.rnn = nn.LSTM(300,
                        hidden_dim,
                        num_layers=n_layers,
                        bidirectional=bidirectional,
                        dropout= dropout if n_layers> 1 else 0)
        elif rnn_type == "GRU":
          self.rnn = nn.GRU(300,
                        hidden_dim,
                        num_layers=n_layers,
                        bidirectional=bidirectional,
                        dropout= dropout if n_layers> 1 else 0)    
        #"""
        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))

        outputs, (hidden) = self.rnn(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions



In [9]:
# Si se mete esta vaina debería quedar bien c:
class GENERAL_NER_CNN(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 dropout,
                 pad_idx):
      
        super().__init__()
        self.embedding = nn.Embedding(
            input_dim,
            embedding_dim,
            padding_idx=pad_idx)
        self.conv = nn.Conv1d(
            in_channels=embedding_dim*input_dim,
            out_channels=hidden_dim,
            kernel_size=3*embedding_dim,
            stride=embedding_dim)
        
        self.pool=nn.MaxPool1d(hidden_dim,stride=embedding_dim)
        
        self.fc=nn.Linear(hidden_dim,output_dim)

        self.drop_out = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)
        #embedded = embedded.view(embedded.shape[0], 1, -1)
        z =nn.ReLU(self.conv(embedded))
        z=self.pool(z)
        
        z=self.fc(z)
        return z

## Parámetros Globales

In [11]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]


--------------------
### Modelo 1

Para el primer modelo escogimos la RNN que estaba definida como Baseline con algunas modificaciones, es decir con una red LSTM, pero cambiando los hiperparametros. Una red LSTM es una arquitectura de RNN, que se caracteriza por ser capaz de conservar los gradientes de las derivadas de las RNN, y esto
lo realiza por un mecanismo de atención, que son unos valores que "definen" cual valor dentro del input, o output tiene más peso, y por lo tanto cuales valores recordar y cuales olvidar.

Como primer hiperparametro se decidió tener el tamaño de las capas ocultas de la RNN, a 5 capas ocultas iniciales. Tiene dimension de Embeddings de 200, y la dimension de sus capas ocultas es de 250. Es una red bidireccional.

En general este modelo tuvo como resultados...


In [ ]:
EMBEDDING_DIM = 200 # b: 200 dimensión de los embeddings. 100--> 200
HIDDEN_DIM = 250  # b: 250 dimensión de la capas LSTM  128 --> 250
N_LAYERS = 8  # b: 5 número de capas. 2 --> 5
DROPOUT = 0.25 # b: 0.25 --> 0.15
BIDIRECTIONAL = True #False --> True

model=GENERAL_NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX, "LSTM")
model_name="Modelo 1"
model_n_epochs = 10 # 10 --> 20
model_loss = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
model_optimizer = optim.Adam(model.parameters())

---------------

### Modelo 2

Para el segundo modelo, se decidió probar con la otra arquitectura conocida de las RNN, la arquitectura GRU, con su implementacion de PyTorch. La arquitectura GRU es conocida al igual que la LSTM, y tambien sirve para el procesamiento del lenguaje. La GRU funciona de manera similar a la LSTM, definiendo un mecanismo de atención para definir cual informacion importante usar versus cual olvidar.

Se decidió la dimensión de los embeddings en 300 y en dos capas ocultas. Se decide tambien definirla como una red bidireccional. Tambien se decidió en procesarlo en 5 epochs, para evitar el overfitting.

In [ ]:
EMBEDDING_DIM = 300 # Base: 300 dimensión de los embeddings. 
HIDDEN_DIM = 128  # Base: 128 dimensión de la capas GRU
N_LAYERS = 2  # Base: 2 número de capas.
DROPOUT = 0.25 # Base: 0.25
BIDIRECTIONAL = True #False --> True
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

model=GENERAL_NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX, "GRU")
model_name="Modelo 2"
model_n_epochs = 5
model_loss = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
model_optimizer = optim.Adam(model.parameters())

---------------


### Modelo 3

Para el modelo 3 se decidió algo distinto, en vez de utilizar una RNN, se decidió utilizar una red convolucional o CNN. Las redes convolucionales son un tipo de red neuronal que se define en base a darle un peso a cada palabra o n-grama, y luego juntarlos todos, luego en base a un criterio se escoge el mejor valor por columna, ya sea por promedio o maximizando. Esto se hace con el proposito de darle más peso a ciertas palabras.

Para esto se definieron 1 capa de convolución, con una función de activación RELU, y una capa de pooling con max pooling.
Para el embedding se decidió 300, y que fuera bidireccional.

In [ ]:
EMBEDDING_DIM = 150 # dimensión de los embeddings. 100--> 120 (Ext_embedding = 300)
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
N_LAYERS = 2  # número de capas.
DROPOUT = 0.25
BIDIRECTIONAL = True #False --> True
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

model=GENERAL_NER_CNN(INPUT_DIM,EMBEDDING_DIM,HIDDEN_DIM,OUTPUT_DIM,N_LAYERS,DROPOUT,PAD_IDX)
model_name="Modelo 3"
model_n_epochs = 5
model_loss = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
model_optimizer = optim.Adam(model.parameters()) #Probando

### Experimentos

La seccion de Experimentos puede definirse en dos partes: primero los experimentos que se hicieron con el baseline antes de definir los modelos para definir los parametros, y la segunda parte que corresponden a los experimentos con los modelos bien definidos.

#### Experimentos Preliminares.

Los experimentos se hicieron con el proposito de ver el peso que tiene cada parametro, y así considerlo al escribir los modelos de redes neuronales. Se usó una metodología con el proposito de evitar hacer todas las combinaciones de parametros. Esta es que: primero definimos un parametro que vamos a cambiar, si este da una mejora respecto al resultado anterior se cambia y aceptamos el cambio, sino pasamos al siguiente parametro.


Para el baseline nos dió los siguientes resultados:

<table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.220</th>
    <th>0.59</th>
    <th>0.64</th>
    <th>0.59</th>
  </tr>

</table>



Entonces, se escoge el primer parametro: los embeddings, se cambia el valor base de 100 a 120 embeddings, y de 100 a 140 Se obtuvieron los siguientes resultados:

<table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.213</th>
    <th>0.58</th>
    <th>0.64</th>
    <th>0.58</th>
  </tr>

</table>

<table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.215</th>
    <th>0.57</th>
    <th>0.63</th>
    <th>0.57</th>
  </tr>

</table>

Podemos ver que las dimensiones del embedding afectan moderadamente a las prediciones, resultando en una función de Loss mas baja.


Escogemos el siguiente parametro las epochs, cambiamos el valor de 10 a 14. asumiendo los embeddings. Se obtuvieron los siguientes resultados:

<table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.217</th>
    <th>0.58</th>
    <th>0.64</th>
    <th>0.58</th>
  </tr>

</table>

Se puede ver que no afecta considerablemente las puntuaciones, respecto al valor anterior, dado que la iteración que minimiza la loss mayoritariamente se encuentra dentro del rango de 2 a 5 epochs.

Escogemos otro parametro, cambiamos la red a una red bidireccional, asumiendo los embeddings. Aqui se tienen los resultados:

<table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.184</th>
    <th>0.59</th>
    <th>0.66</th>
    <th>0.59</th>
  </tr>

</table>

Aquí se nota una mejoría considerable respecto a los valores previos, bajando el valor de Loss a menos de 0.200. Por lo que se puede ver que la bidireccionalidad tiene un peso considerable en el rendimiento de la red.

Cambiando la cantidad de caps ocultas el resultado tampoco cambia mucho, pero a la red no le perjudica tener más capas, mantienendo los mismos valores con 2, 3 y 4 capas.

Respecto a la función de Loss, la funcion de Cross Entropy Loss es la que presenta los mejores resultados, los otros de Loss muy altas para considerarse.
Lo mismo con el optimizador Adam, al cambiar de optimizador los resultados empeoraban por lo que se decidió mantener el optimizador para todos los modelos.


### LSTM
Parámetros base:
  <table style="width:100%">
  <tr>
  <th>EMBEDDING_DIM</th>
  <th>HIDDEN_DIM</th>
  <th>N_LAYERS</th>
  <th>DROPOUT</th>
  <th>BIDIRECTIONAL</th>
  </tr>
  <tr>
    <th>200</th>
    <th>250</th>
    <th>5</th>
    <th>0.25</th>
    <th>True</th>
  </tr>
  </table>
  - Resultados parámetros base:
    <table style="width:100%">
  <tr> 
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.203</th>
    <th>0.62</th>
    <th>0.66</th>
    <th>0.64</th>
  </tr>
</table>
  - Dimensión de Embeddings: $200 \to 100$  
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.236</th>
    <th>0.53</th>
    <th>0.58</th>
    <th>0.56</th>
  </tr>
  </table>
  - Dimensión de Embeddings: $200 \to 300$ 
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.238</th>
    <th>0.53</th>
    <th>0.58</th>
    <th>0.56</th>
  </tr>
  </table>
  - Dimensión de Capa Oculta: $250 \to 150$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.237</th>
    <th>0.55</th>
    <th>0.60</th>
    <th>0.56</th>
  </tr>
  </table>
  - Dimensión de Capa Oculta: $250 \to 350$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.695</th>
    <th>0.08</th>
    <th>0.23</th>
    <th>0.06</th>
  </tr>
  </table>

  </table>
  - Layers: $5 \to 1$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.185</th>
    <th>0.63</th>
    <th>0.69</th>
    <th>0.62</th>
  </tr>
</table>
  - Layers: $5 \to 8$ 
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.249</th>
    <th>0.48</th>
    <th>0.54</th>
    <th>0.50</th>
  </tr>
</table>
  - Dropout: $25\% \to 10\%$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.</th>
    <th>0.</th>
    <th>0.</th>
    <th>0.</th>
  </tr>
  </table>
  - Dropout:  $25\% \to 50\%$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.303</th>
    <th>0.59</th>
    <th>0.64</th>
    <th>0.61</th>
  </tr>
  </table>
</table>
  - Bi Directional: $\to$ False
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.226</th>
    <th>0.57</th>
    <th>0.61</th>
    <th>0.59</th>
  </tr>
</table>



[Comentarios]

### GRU:

Parámetros base:
  <table style="width:100%">
  <tr>
  <th>EMBEDDING_DIM</th>
  <th>HIDDEN_DIM</th>
  <th>N_LAYERS</th>
  <th>DROPOUT</th>
  <th>BIDIRECTIONAL</th>
  </tr>
  <tr>
    <th>300</th>
    <th>128</th>
    <th>2</th>
    <th>0.25</th>
    <th>True</th>
  </tr>
  </table>
  - Resultados parámetros base:
    <table style="width:100%">
  <tr> 
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.184</th>
    <th>0.62</th>
    <th>0.68</th>
    <th>0.61</th>
  </tr>
  </table>
  - Dimensión de Embeddings: $300 \to 150$  
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.183</th>
    <th>0.60</th>
    <th>0.67</th>
    <th>0.60</th>
  </tr>
  </table>
  - Dimensión de Embeddings: $300 \to 450$ 
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.184</th>
    <th>0.61</th>
    <th>0.67</th>
    <th>0.60</th>
  </tr>
  </table>
  - Dimensión de Capa Oculta: $128 \to 192$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.193</th>
    <th>0.60</th>
    <th>0.67</th>
    <th>0.59</th>
  </tr>
  </table>
  - Dimensión de Capa Oculta: $128 \to 64$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.180</th>
    <th>0.59</th>
    <th>0.66</th>
    <th>0.59</th>
  </tr>
  </table>
</table>
  - Dropout: $\to 0.10\%$:
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.179</th> 
    <th>0.62</th>
    <th>0.68</th>
    <th>0.61</th>
  </tr>
</table>
  - Dropout: $\to 0.50\%$
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.185</th>
    <th>0.62</th>
    <th>0.68</th>
    <th>0.62</th>
  </tr>
</table>
</table>
  - Layers: $\to$ 1
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr> 
    <th>0.186</th>
    <th>0.60</th>
    <th>0.66</th>
    <th>0.60</th>
  </tr>
</table>
  - Layers: $\to$ 6
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.216</th>
    <th>0.58</th>
    <th>0.63</th>
    <th>0.59</th>
  </tr>
</table>
</table>
  - Bi Directional: $\to$ False
  <table style="width:100%">
  <tr>
    <th>Val. Loss</th>
    <th>Val.  f1</th>
    <th>Val. Precision </th>
    <th>Val. Recall</th>
  </tr>
  <tr>
    <th>0.211</th>
    <th>0.59</th>
    <th>0.66</th>
    <th>0.59</th>
  </tr>
</table>

[Comentarios]

### CNN:
    - ...


#### Entrenamiento de la red





In [ ]:
# Reasignamos y reiniciamos 
model_name = model_name
criterion = model_loss
n_epochs = model_n_epochs
optimizer=model_optimizer

model.apply(init_weights)

# Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
model = model.to(device)
criterion = criterion.to(device)

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)

    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))
# Limpiar ram de cuda
torch.cuda.empty_cache()

valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'FINAL RESULT: Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.284 | Train f1: 0.40 | Train precision: 0.49 | Train recall: 0.37
	 Val. Loss: 0.197 |  Val. f1: 0.56 |  Val. precision: 0.64 | Val. recall: 0.55
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.082 | Train f1: 0.72 | Train precision: 0.76 | Train recall: 0.72
	 Val. Loss: 0.184 |  Val. f1: 0.62 |  Val. precision: 0.68 | Val. recall: 0.61
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.038 | Train f1: 0.85 | Train precision: 0.86 | Train recall: 0.84
	 Val. Loss: 0.216 |  Val. f1: 0.61 |  Val. precision: 0.68 | Val. recall: 0.60
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.024 | Train f1: 0.89 | Train precision: 0.91 | Train recall: 0.89
	 Val. Loss: 0.230 |  Val. f1: 0.61 |  Val. precision: 0.68 | Val. recall: 0.61
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.017 | Train f1: 0.93 | Train precision: 0.93 | Train recall: 0.92
	 Val. Loss: 0.239 |  Val. f1: 0.62 |  Val. precision: 0.69 | Val. recall: 0.61
FINAL RESULT: Val. Loss: 0.184 |  Val. f

#### Evaluamos el set de validación con el modelo final

Estos son los resultados de predecir el dataset de evaluación con el *mejor* modelo entrenado.

Val. Loss: 0.303 |  Val. f1: 0.59 | Val. precision: 0.64 | Val. recall: 0.61



### Predecir datos para la competencia

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, predeciremos las etiquetas que serán evaluadas en la competencia.

In [ ]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])


    return predictions


predictions = predict_labels(model, test_iterator, criterion)

### Generar el archivo para la submission

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [ ]:
import os, shutil

if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for word, tag in predictions:
    f.write(word + ' ' + tag + '\n')
f.write('\n')
f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')

In [ ]:
# A veces no funciona a la primera. Ejecutar mas de una vez para obtener el archivo...
from google.colab import files
files.download('predictions.zip')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Conclusiones



...